In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.sparse.linalg import svds
from scipy.spatial.distance import cosine 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [25]:
path = '/Users/syedfaquaruddinquadri/Documents/Applied ML and DM/Final Project/machine-learning-dse-i210-final-project-yelprestaurantrecommender/data/processed'
business = pd.read_csv(path + '/business.csv')
restaurants = business[business.categories.fillna('-').str.lower().str.contains('restaurant')]

In [26]:
business.head()

,Unnamed: 0,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,0,712 Adams St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",M0XSSHqrASOnhgbWDJIpQA,"Vape Shops, Tobacco Shops, Personal Shopping, ...",New Orleans,"Row(Friday='10:0-19:0', Monday='10:0-19:0', Sa...",1,29.941468,-90.129953,Herb Import Co,70118.0,5,4.0,LA
1,1,100 Iberville St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",w_AMNoI1iG9eay7ncmc67w,"Event Planning & Services, Hotels, Hotels & Tr...",New Orleans,NaN,1,29.951359,-90.064672,River 127,70130.0,12,3.0,LA
2,2,723 St Peter St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",uczmbBk5O3tYhGue13dCDg,"Hotels & Travel, Tours, Local Flavor",New Orleans,"Row(Friday='10:0-20:0', Monday='0:0-0:0', Satu...",1,29.958431,-90.065173,New Orleans Spirit Tours,70130.0,38,4.0,LA
3,3,1001 Poydras St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",YNjyv0gfOr2g8lbmUpTnKg,"Nightlife, Pubs, Event Planning & Services, Wi...",New Orleans,"Row(Friday='11:0-23:0', Monday='0:0-0:0', Satu...",1,29.950647,-90.074427,Copper Vine,70112.0,350,4.5,LA
4,4,600 Decatur St,"Row(AcceptsInsurance=None, AgesAllowed=None, A...",J_ksUDPpzPwfTGtI4zTRnQ,"Event Planning & Services, Caterers, Party & E...",New Orleans,"Row(Friday='9:0-16:0', Monday='9:0-16:0', Satu...",1,29.955925,-90.062962,Riverview Room,70130.0,7,4.5,LA


In [27]:
restaurants.columns

Index(['Unnamed: 0', 'address', 'attributes', 'business_id', 'categories',
       'city', 'hours', 'is_open', 'latitude', 'longitude', 'name',
       'postal_code', 'review_count', 'stars', 'state'],
      dtype='object')

In [28]:
restaurants = restaurants.drop(['Unnamed: 0','address', 'state', 'postal_code',
       'latitude', 'longitude', 'is_open', 'attributes', 'hours'], axis = 1)


In [29]:
restaurants.shape


(2262, 6)

In [30]:
path1 = '/Users/syedfaquaruddinquadri/Documents/Applied ML and DM/Final Project'

In [31]:
reviews = pd.read_csv(path1  + '/reviews.csv')


/var/folders/6y/65h__9y57mg1xqznxbt3fvtr0000gn/T/ipykernel_41095/1096967167.py:1: DtypeWarning: Columns (0,1,2,3,5,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv(path1  + '/reviews.csv')


In [32]:
reviews.shape

(1095800, 10)

In [33]:
reviews.columns

Index(['Unnamed: 0', 'business_id', 'cool', 'date', 'funny', 'review_id',
       'stars', 'text', 'useful', 'user_id'],
      dtype='object')

In [35]:
reviews.drop('Unnamed: 0', axis =1, inplace = True)
reviews = reviews.drop(['useful', 'funny','cool','date'], axis = 1)

In [36]:
reviews.shape

(1095800, 5)

In [70]:
nola = pd.merge(reviews, restaurants, on='business_id')

In [71]:
nola.shape

(536257, 10)

In [72]:
nola.columns

Index(['business_id', 'review_id', 'stars_x', 'text', 'user_id', 'categories',
       'city', 'name', 'review_count', 'stars_y'],
      dtype='object')

In [73]:
nola = nola.drop_duplicates(['user_id', 'business_id', 'stars_y'], keep='first')

In [74]:
nola.shape

(466149, 10)

In [75]:
user_counts = nola['user_id'].value_counts().to_dict()
user_list = []

In [78]:
len(user_counts)

201869

In [79]:
sorted(user_counts.values())[-20:]

[175,
 178,
 178,
 180,
 186,
 188,
 191,
 195,
 200,
 219,
 220,
 223,
 230,
 236,
 239,
 441,
 523,
 525,
 600,
 662]

In [80]:
for key in user_counts.keys(): 
    if user_counts[key] >= 50: 
        user_list.append(key)

In [81]:
filtered_nola = nola[nola['user_id'].isin(user_list)]
print('Data shape after filtering by users: ', filtered_nola.shape)

Data shape after filtering by users:  (28808, 10)


In [82]:
print('Unique users: ', len(filtered_nola['user_id'].unique()))

Unique users:  310
